In [245]:
%matplotlib inline
import matplotlib
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random
import itertools as it

from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [246]:
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/submit/'

In [7]:
train_files_original = glob('/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/train_original/*/*.npz')
print(len(train_files_original))

train_files_pseudo = glob('/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/train_pseudo/*/*.npz')
print(len(train_files_pseudo))

f2c = {}
for fname in it.chain(train_files_original, train_files_pseudo):
    f2c[fname] = fname.split('/')[-2]
    
f2c_presudo = {}
for fname in train_files_pseudo:
    f2c_presudo[fname] = fname.split('/')[-2]

test_files = glob('/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/test/*.npz')
print(len(test_files))

49500
1602
2640


In [13]:
pd.Series(list(f2c_presudo.values())).value_counts()

Motorola-Nexus-6        232
Motorola-Droid-Maxx     210
Samsung-Galaxy-S4       205
iPhone-6                195
Samsung-Galaxy-Note3    181
Motorola-X              177
HTC-1-M7                166
Sony-NEX-7              127
LG-Nexus-5x              77
iPhone-4s                32
dtype: int64

In [80]:
m_pseudo_avg = []
m_pseudo_max = []
files = []
y = []

for fname, c in f2c_presudo.items():
    m_pseudo_avg.append(np.load(fname)['f_mean'])
    m_pseudo_max.append(np.load(fname)['f_max'])
    files.append(os.path.basename(fname)[:-4])
    y.append(c)
    
# le = LabelEncoder().fit(y)
# y = le.transform(y)

m_pseudo_avg = np.array(m_pseudo_avg)
m_pseudo_max = np.array(m_pseudo_max)

In [153]:
k = 23
pred = {}

keys = ['ae', 'me', 'ac', 'mc']

n_m_pseudo_avg = (m_pseudo_avg/np.sqrt((m_pseudo_avg**2).sum(axis=1))[:, np.newaxis])
n_m_pseudo_max = (m_pseudo_max/np.sqrt((m_pseudo_max**2).sum(axis=1))[:, np.newaxis])

for fname in tqdm_notebook(test_files):
    key = os.path.basename(fname)[:-4]
    
    f_avg = np.load(fname)['f_mean']
    f_max = np.load(fname)['f_max']
    
    n_f_avg = np.sqrt((f_avg**2).sum())
    n_f_max = np.sqrt((f_max**2).sum())
    
    d_avg_euc = ((m_pseudo_avg - f_avg)**2).sum(axis=1)
    d_max_euc = ((m_pseudo_max - f_max)**2).sum(axis=1)
    
    d_avg_cos = 1 - np.dot((f_avg/n_f_avg), n_m_pseudo_avg.T)
    d_max_cos = 1 - np.dot((f_max/n_f_max), n_m_pseudo_max.T)
    
    df = pd.DataFrame(
        list(zip(files, y, d_avg_euc, d_max_euc, d_avg_cos, d_max_cos)), 
        columns=['f', 'y', 'ae', 'me', 'ac', 'mc'])
    
    pred[key] = {}
    
    for d in keys:
        pred[key][d] = list(df.sort_values([d])['y'].head(k).value_counts(normalize=True).items())

In [188]:
votes = {}

for k, d in pred.items():
    votes[k] = []
    for _, v in d.items():
        votes[k].append(v[0][0])

In [204]:
res = []

for k, d in pred.items():
    vtmp = []
    for _, v in d.items():
        vtmp.append(v[0])        
    res.append((k, sorted(vtmp, key=lambda t: t[1], reverse=True)[0][0]))

In [155]:
res = []

for k, v in votes.items():
    res.append((k, list(pd.Series(v).value_counts().items())[0][0]))

In [205]:
(pd.Series([c for (k, c) in res]).value_counts() - 264).abs().sum()/len(test_files)

0.16136363636363638

In [238]:
unalt = []
manip = []
for f, d in pred.items():
    if 'unalt' in f:
        cur = unalt
    else:
        cur = manip
    
    p = []
    for c in set(it.chain(*[[c for (c, _) in v] for (k, v) in d.items()])):
        p.append((c, np.mean([(dict(v)[c] if c in dict(v) else 0) for (k, v) in d.items()])))
    for c in set(le.classes_).difference([k for (k, v) in p]):
        p.append((c, 0.0))
    # cur.append((f, sorted(p, key=lambda t: t[1], reverse=True)[0][0]))
    cur.append((f, p))

In [240]:
unalt_tmp = unalt.copy()
manip_tmp = manip.copy()
print(len(unalt_tmp), len(manip_tmp))

t = int(0.1*1320)
print(t)

res = []

c_counts = defaultdict(int)
for _ in range(1320):
    skip_f = set([k for (k, _) in res])
    unalt_tmp = [(f, sorted(p, key=lambda t: t[-1], reverse=True)) for (f, p) in unalt_tmp if f not in skip_f]
    unalt_tmp = sorted(unalt_tmp, key=lambda t: t[-1][0], reverse=True)
    f, p = unalt_tmp[0]
    c, _ = p[0]
    res.append((f, c))
    c_counts[c] += 1
    
    if c_counts[c] == t:
        unalt_tmp = [(f, [(k, v) for (k, v) in p if k != c]) for (f, p) in unalt_tmp]
        
c_counts = defaultdict(int)
for _ in range(1320):
    skip_f = set([k for (k, _) in res])
    manip_tmp = [(f, sorted(p, key=lambda t: t[-1], reverse=True)) for (f, p) in manip_tmp if f not in skip_f]
    manip_tmp = sorted(manip_tmp, key=lambda t: t[-1][0], reverse=True)
    f, p = manip_tmp[0]
    c, _ = p[0]
    res.append((f, c))
    c_counts[c] += 1
    
    if c_counts[c] == t:
        manip_tmp = [(f, [(k, v) for (k, v) in p if k != c]) for (f, p) in manip_tmp]
        
print(len(res))

1320 1320
132
2640


In [247]:
with open(os.path.join(out_dir, 'submit__unalt.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_unalt' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit__manip.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_manip' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        f.write('%s,%s\n' % (fname, c))